## Setup: Random Seeds

In [1]:
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

print(f"PyTorch version: {torch.__version__}")
print(f"Device: {torch.device('cuda' if torch.cuda.is_available() else 'cpu')}")

PyTorch version: 2.8.0+cu128
Device: cuda


## Import Libraries

In [2]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os

---
# QUESTION 1: Which loss function to use?

**Options:**
- `nn.MSELoss()`
- `nn.BCEWithLogitsLoss()`
- `nn.CrossEntropyLoss()`
- `nn.CosineEmbeddingLoss()`

** Answer:** `nn.BCEWithLogitsLoss()` BCEWithLogitsLoss (Binary Cross Entropy with Logits)

Used for binary classification (2 classes)
Expects 1 output neuron
Applies sigmoid internally

In [3]:
class HairTypesCNN(nn.Module):
    def __init__(self):
        super(HairTypesCNN, self).__init__()
        
        # Input: (3, 200, 200)
        # Conv2d: 32 filters, kernel 3x3, padding=0, stride=1
        self.conv1 = nn.Conv2d(3, 32, kernel_size=(3, 3), padding=0, stride=1)
        self.relu1 = nn.ReLU()
        
        # MaxPool2d: 2x2
        self.maxpool = nn.MaxPool2d(kernel_size=(2, 2))
        
        # After Conv2d(3x3, no padding): (200-3+1) = 198 -> (32, 198, 198)
        # After MaxPool2d(2x2): 198/2 = 99 -> (32, 99, 99)
        # Flattened: 32 * 99 * 99 = 313632
        self.flatten_size = 32 * 99 * 99
        
        # Linear layer: 64 neurons with ReLU
        self.fc1 = nn.Linear(self.flatten_size, 64)
        self.relu2 = nn.ReLU()
        
        # Output layer: 1 neuron for binary classification
        self.fc2 = nn.Linear(64, 1)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool(x)
        x = x.view(-1, self.flatten_size)
        x = self.fc1(x)
        x = self.relu2(x)
        x = self.fc2(x)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HairTypesCNN().to(device)
print("Model created successfully")
print(model)

Model created successfully
HairTypesCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (relu1): ReLU()
  (maxpool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=313632, out_features=64, bias=True)
  (relu2): ReLU()
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)


---
# QUESTION 2: Number of parameters

What's the total number of parameters of the model?

**Answer:** `20073473`

In [4]:
# Option 1: Using torchsummary (uncomment if installed)
from torchsummary import summary
summary(model, (3, 200, 200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 198]             896
              ReLU-2         [-1, 32, 198, 198]               0
         MaxPool2d-3           [-1, 32, 99, 99]               0
            Linear-4                   [-1, 64]      20,072,512
              ReLU-5                   [-1, 64]               0
            Linear-6                    [-1, 1]              65
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 21.54
Params size (MB): 76.57
Estimated Total Size (MB): 98.57
----------------------------------------------------------------


In [ ]:
# Option 2: Manual counting
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 20073473


## Data Preparation

In [ ]:
from torchvision.datasets import ImageFolder

train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

test_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

train_dataset = ImageFolder('./data/train', transform=train_transforms)
validation_dataset = ImageFolder('./data/test', transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True, num_workers=2)
validation_loader = DataLoader(validation_dataset, batch_size=20, shuffle=False, num_workers=2)

class_names = train_dataset.classes
print(f"Class names: {class_names}")
print(f"Train size: {len(train_dataset)}")
print(f"Validation size: {len(validation_dataset)}")

## Training Loop

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

---
# QUESTION 3: Median of training accuracy

What is the median of training accuracy for all the epochs for this model?

In [ ]:
import numpy as np
median_acc = np.median(history['acc'])
print(f"Median Training Accuracy: {median_acc:.4f}")

---
# QUESTION 4: Standard deviation of training loss

What is the standard deviation of training loss for all the epochs for this model?

In [ ]:
std_loss = np.std(history['loss'])
print(f"Standard Deviation of Training Loss: {std_loss:.4f}")

## Data Augmentation

In [ ]:
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

train_dataset = ImageFolder('./data/train', transform=train_transforms)
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True, num_workers=2)

print("Data augmentation applied to training set.")

## Continue Training

In [ ]:
# Continue training for 10 more epochs
num_epochs = 10
# We will store the history for these new epochs separately to answer Q5 and Q6 easily
history_aug = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history_aug['loss'].append(epoch_loss)
    history_aug['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history_aug['val_loss'].append(val_epoch_loss)
    history_aug['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+11}/{num_epochs+10}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

---
# QUESTION 5: Mean of test loss

What is the mean of test loss for all the epochs for the model trained with augmentations?

In [ ]:
mean_val_loss_aug = np.mean(history_aug['val_loss'])
print(f"Mean Validation Loss (Augmented): {mean_val_loss_aug:.4f}")

---
# QUESTION 6: Average of test accuracy for the last 5 epochs

What's the average of test accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?

In [ ]:
avg_val_acc_last5 = np.mean(history_aug['val_acc'][-5:])
print(f"Average Validation Accuracy (Last 5 epochs): {avg_val_acc_last5:.4f}")